<a href="https://colab.research.google.com/github/Matthew0077/genai/blob/main/HW13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
import openai
print(openai.__version__)

1.81.0


In [ ]:
!pip install --upgrade openai ipywidgets sqlalchemy pandas matplotlib
%load_ext google.colab.data_table
%load_ext widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uni

In [ ]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import openai
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
print("OpenAI version:", openai.__version__)

OpenAI version: 1.81.0


In [ ]:
openai.api_key = os.getenv("OpenAI")

In [ ]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS entries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    created_at TEXT NOT NULL,
    content TEXT NOT NULL,
    anxiety_score REAL,
    ai_feedback TEXT
)
''')
conn.commit()

In [ ]:
def analyze_entry(text: str) -> tuple[float, str]:
    prompt = f"""
你是一位教育心理學專家，專門分析使用者的焦慮情緒。請依據以下日記文字：
---
{text}
---
1. 以 0 到 100 的尺度給出「焦慮指數」，0 表示毫無焦慮，100 表示極度焦慮。
2. 給出一段 2～3 句的「調節建議」。
請用以下格式回覆：
焦慮指數：<數值>
調節建議：<文字>
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "你是教育心理學專家。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=150
    )
    reply = response.choices[0].message.content.strip()
    score = 0.0
    feedback = ""
    for line in reply.splitlines():
        if line.startswith("焦慮指數"):
            try:
                score = float(line.split("：")[1].strip())
            except:
                score = 0.0
        elif line.startswith("調節建議"):
            feedback = line.split("：", 1)[1].strip()
    return score, feedback


In [ ]:
textarea = widgets.Textarea(
    value='',
    placeholder='請在此輸入今天的焦慮日記…',
    description='日記輸入：',
    layout=widgets.Layout(width='80%', height='150px')
)
submit_btn = widgets.Button(
    description='送出並分析',
    button_style='primary'
)
output = widgets.Output()


In [ ]:
def on_submit_clicked(b):
    with output:
        clear_output()
        user_text = textarea.value.strip()
        if not user_text:
            print("❗ 請先輸入日記內容。")
            return
        # 呼叫 GPT-4 分析
        score, feedback = analyze_entry(user_text)
        # 把結果寫進 anxiety_journal.db
        cursor.execute(
            'INSERT INTO entries (created_at, content, anxiety_score, ai_feedback) VALUES (?, ?, ?, ?)',
            (datetime.utcnow().strftime("%Y-%m-%d %H:%M"), user_text, score, feedback)
        )
        conn.commit()
        # 顯示當次分析結果
        print(f"✅ 已儲存。焦慮指數：{score:.1f}")
        print(f"🤖 AI 建議：{feedback}\n")

        # 讀取並顯示「完整歷史」所有筆記
        df_all = pd.read_sql_query('SELECT * FROM entries ORDER BY created_at ASC', conn)
        print("--- 歷史所有日記紀錄 ---")
        display(df_all)

        # 繪製「所有筆記」的焦慮指數折線圖
        df_all['created_at'] = pd.to_datetime(df_all['created_at'])
        plt.figure(figsize=(6, 3))
        plt.plot(df_all['created_at'], df_all['anxiety_score'], marker='o', linestyle='-')
        plt.xlabel("日期")
        plt.ylabel("焦慮指數")
        plt.title("焦慮指數折線圖（歷史）")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [ ]:
submit_btn.on_click(on_submit_clicked)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()